<a href="https://colab.research.google.com/github/amitaipat-create/Session2/blob/main/Mission_2_Practice_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agents with memory, conversation history, multi-turn conversations

In [ ]:
%pip install langchain==0.3.27 langchain-openai==0.3.29 langchain-community==0.3.27 pydantic==2.11.10 requests==2.32.4 --quiet

In [ ]:
import os
import getpass
from dotenv import load_dotenv

# Load environment variables from .env file (for local development)
load_dotenv()

# OpenAI API Key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory

try:
    from IPython.display import display, Markdown
except ImportError:
    def display(x): print(x)
    def Markdown(x): return x

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [ ]:
# ============================================================================
# SETUP: Create Simple Tools
# ============================================================================

@tool
def get_weather(city: str) -> str:
    """Get weather information for a city"""
    # Mock weather data (in real app, call weather API)
    weather_data = {
        "paris": "Sunny, 22°C",
        "london": "Cloudy, 15°C",
        "tokyo": "Rainy, 18°C",
        "new york": "Sunny, 25°C"
    }
    city_lower = city.lower()
    return weather_data.get(city_lower, f"Weather data not available for {city}")

@tool
def get_population(city: str) -> str:
    """Get population of a city"""
    # Mock population data (in real app, call API)
    population_data = {
        "paris": "2.1 million",
        "london": "9.0 million",
        "tokyo": "14.0 million",
        "new york": "8.5 million"
    }
    city_lower = city.lower()
    return population_data.get(city_lower, f"Population data not available for {city}")

tools = [get_weather, get_population]

In [ ]:
# ============================================================================
# CONCEPT 1: Agent Without Memory (Loses Context)
# ============================================================================

# Agent without memory forgets previous conversations
# Each question is treated independently

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools to answer questions."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor_no_memory = AgentExecutor(agent=agent, tools=tools, verbose=False)

# First question
result1 = agent_executor_no_memory.invoke({"input": "What's the weather in Paris?"})
print("Question 1: What's the weather in Paris?")
print(f"Answer: {result1['output']}")

# Follow-up question (agent doesn't remember Paris was mentioned)
result2 = agent_executor_no_memory.invoke({"input": "What about the population?"})
print("\nQuestion 2: What about the population?")
print(f"Answer: {result2['output']}")

# Problem: Agent doesn't know "the population" refers to Paris!

In [ ]:
# ============================================================================
# CONCEPT 2: Agent With Memory (Remembers Context)
# ============================================================================

# Memory stores conversation history so agent remembers previous messages
# This enables natural follow-up questions

# Create memory to store conversation
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Update prompt to include chat history
prompt_with_memory = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools to answer questions. Remember previous conversation."),
    ("placeholder", "{chat_history}"),  # Memory goes here
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent_with_memory = create_tool_calling_agent(llm, tools, prompt_with_memory)

# Create executor with memory
agent_executor = AgentExecutor(
    agent=agent_with_memory,
    tools=tools,
    memory=memory,
    verbose=False
)

# First question
result1 = agent_executor.invoke({"input": "What's the weather in Paris?"})
print("\n" + "=" * 60)
print("WITH MEMORY:")
print("=" * 60)
print("\nQuestion 1: What's the weather in Paris?")
print(f"Answer: {result1['output']}")

# Follow-up question (agent remembers Paris from previous question!)
result2 = agent_executor.invoke({"input": "What about the population?"})
print("\nQuestion 2: What about the population?")
print(f"Answer: {result2['output']}")

# Success! Agent knows "the population" refers to Paris!

In [ ]:
# ============================================================================
# CONCEPT 3: Multi-Turn Conversation
# ============================================================================

# Reset memory for clean example
memory2 = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor2 = AgentExecutor(
    agent=create_tool_calling_agent(llm, tools, prompt_with_memory),
    tools=tools,
    memory=memory2,
    verbose=False
)

# Natural conversation flow
print("\n" + "=" * 60)
print("MULTI-TURN CONVERSATION:")
print("=" * 60)

# Turn 1
response1 = agent_executor2.invoke({"input": "Tell me about London"})
print("\nYou: Tell me about London")
print(f"Agent: {response1['output']}")

# Turn 2 - Follow-up question
response2 = agent_executor2.invoke({"input": "What's the weather there?"})
print("\nYou: What's the weather there?")
print(f"Agent: {response2['output']}")

# Turn 3 - Another follow-up
response3 = agent_executor2.invoke({"input": "And how many people live there?"})
print("\nYou: And how many people live there?")
print(f"Agent: {response3['output']}")

In [ ]:
# Notice: Agent understands "there" refers to London from the conversation!

# ============================================================================
# EXERCISES
# ============================================================================

# EXERCISE 1: Try a conversation without memory
# Create a new agent without memory and ask:
# "What's the weather in Tokyo?" then "What about the population?"
# See how it fails to understand the follow-up

# EXERCISE 2: Try a conversation with memory
# Create a new agent with memory and ask the same questions
# See how it remembers context!

# EXERCISE 3: Build a multi-turn conversation
# Ask about a city, then ask follow-up questions using "it", "there", "that city"
# Watch how the agent maintains context throughout the conversation

display(Markdown("## ✓ Practice Complete!"))
display(Markdown("You learned: Agents with memory, conversation history, multi-turn conversations"))